In [1]:
# Libraries for building network
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Libraries for dataset
import torchvision
import torchvision.transforms as transforms
from torchvision import models

# Miscellaneous Libraries
import time

In [12]:
# Number of epochs for training
num_epochs = 1

# Batch Size for training/testing
batch_size = 5

# Learning Rate for optimizer
learning_rate = 0.03

# Dimensions of model (ResNet-18)
dim = 224

In [13]:
# Transformation for training data
transform_train = torchvision.transforms.Compose([
    transforms.RandomResizedCrop(dim),
    transforms.ToTensor(),                            # Convert grayscale image to pytorch tensor
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),             # Normalize grayscale data
])

# Transformation for training data
transform_test = transforms.Compose([
    transforms.Resize(256),
    transforms.CenterCrop(224),
    transforms.ToTensor(),                            # Convert grayscale image to pytorch tensor
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),             # Normalize grayscale data
])


In [14]:
# Download training data
trainset = torchvision.datasets.CIFAR10(root='./files', train=True, download=True, 
                                      transform=transform_train)

# Initialize dataloader for training data
train_loader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, 
                                           num_workers=8)

# Download testing Data
testset = torchvision.datasets.CIFAR10(root='./files', train=False, download=False, 
                                     transform=transform_test)

# Initialize dataloader for testing data
test_loader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, 
                                          num_workers=8)

Files already downloaded and verified


In [15]:
# Initialize pretrained ResNet-18 model
model = torchvision.models.resnet18(pretrained=True)

# Turn of gradient calculation for all layers (We don't train them)
for param in model.parameters():
    param.requires_grad = False

# Create new final fully-connected layer for our problem
num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 10)                                             

# Definie loss function (Cross Entropy Loss)
criterion = nn.CrossEntropyLoss()                                      

# Initialize Optimizer (ADAM)
optimizer = torch.optim.Adam(model.fc.parameters(), lr=learning_rate)     

# Set model to training (updating weights)
model.train();

In [16]:
# Store time to calculate train time
start_time = time.time()

# Store loss and accuracy data
loss = []
accuracy = []

# Train the model
# Loop for number of epochs
for epoch in range(num_epochs):
    # Loop through data in batch sized increments
    for batch_idx, (X_train_batch, Y_train_batch) in enumerate(train_loader):
        # If trained on all data in epoch, move onto next epoch
        if(Y_train_batch.shape[0]<batch_size):
            continue

        # Forward pass through network
        output = model(X_train_batch)                           
        # Calculate loss of predictions
        curr_loss = criterion(output, Y_train_batch)            
        # Store loss
        loss.append(curr_loss.item())                           

        # Clear last calculation
        optimizer.zero_grad()                                   
        # Calculate gradient based on loss
        curr_loss.backward()                                    
        # Update model weights
        optimizer.step()                                        

        # Extract model predictions
        _, predicted = torch.max(output.data, 1) 
        # Calculate number of correct predictions
        correct = (predicted == Y_train_batch).sum().item()     
        # Calculate/store accuracy
        accuracy.append(correct/Y_train_batch.size(0))          
        
        # Intermitently print statistics
        if batch_idx % 100 == 0:
            print('Epoch: ' + str(epoch+1) + '/' + str(num_epochs) + ', Step: ' 
                  + str(batch_idx+1) + '/' + str(len(train_loader)) + ', Loss: ' 
                  + str(curr_loss.item()) + ', Accuracy: ' 
                  + str(correct/Y_train_batch.size(0)*100) + '%')

# Store time to calculate train time
end_time = time.time()

# Print train time
print('Run Time: ' + str(end_time - start_time))

Epoch: 1/1, Step: 1/10000, Loss: 2.6219639778137207, Accuracy: 0.0%
Epoch: 1/1, Step: 101/10000, Loss: 2.95889949798584, Accuracy: 60.0%
Epoch: 1/1, Step: 201/10000, Loss: 4.895359039306641, Accuracy: 20.0%
Epoch: 1/1, Step: 301/10000, Loss: 0.2776709496974945, Accuracy: 80.0%
Epoch: 1/1, Step: 401/10000, Loss: 10.29859733581543, Accuracy: 40.0%
Epoch: 1/1, Step: 501/10000, Loss: 18.43816566467285, Accuracy: 0.0%
Epoch: 1/1, Step: 601/10000, Loss: 11.276357650756836, Accuracy: 60.0%
Epoch: 1/1, Step: 701/10000, Loss: 10.3700590133667, Accuracy: 20.0%
Epoch: 1/1, Step: 801/10000, Loss: 15.29130744934082, Accuracy: 0.0%
Epoch: 1/1, Step: 901/10000, Loss: 4.639434337615967, Accuracy: 60.0%
Epoch: 1/1, Step: 1001/10000, Loss: 7.205655097961426, Accuracy: 0.0%
Epoch: 1/1, Step: 1101/10000, Loss: 3.1851916313171387, Accuracy: 40.0%
Epoch: 1/1, Step: 1201/10000, Loss: 2.5422613620758057, Accuracy: 60.0%
Epoch: 1/1, Step: 1301/10000, Loss: 6.182482719421387, Accuracy: 60.0%
Epoch: 1/1, Step: 1

In [17]:
# Test the model
# Set model to testing (constant weights)
model.eval()

with torch.no_grad():
    # Store number of correct/total samples in test data
    correct = 0
    total = 0
    
    # Loop through test data
    for X_test_batch, Y_test_batch in test_loader:
        # Forward pass through network
        output = model(X_test_batch)  
        
        # Extract prediction
        _, predicted = torch.max(output.data, 1)    
        
        # Update total number of sample
        total += Y_test_batch.size(0)  
        
        # Update number of correct predictions
        correct += (predicted == Y_test_batch).sum().item()     

print('Test Accuracy: ' + str((correct/total) * 100) + '%')

Test Accuracy: 64.12%
